In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv("./data/parsed/parsed_emails.csv")

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


In [7]:
df

,Employee,EmailContent,sender_email,subject,num_receivers,email_length,email_domain,sent_time,is_forwarded
0,arnold-j,Message-ID: <19171686.1075857585034.JavaMail.e...,slafontaine@globalp.com,re:summer inverses,1.0,29.0,globalp.com,"Fri, 8 Dec 2000 05:05:00 -0800 (PST)",0.0
1,arnold-j,Message-ID: <29887033.1075857630725.JavaMail.e...,iceoperations@intcx.com,The WTI Bullet swap contracts,2.0,110.0,intcx.com,"Tue, 15 May 2001 09:43:00 -0700 (PDT)",0.0
2,arnold-j,Message-ID: <29084893.1075849630138.JavaMail.e...,jeff.youngflesh@enron.com,Invitation: EBS/GSS Meeting w/Bristol Babcock ...,4.0,17.0,enron.com,"Mon, 27 Nov 2000 01:49:00 -0800 (PST)",0.0
3,arnold-j,Message-ID: <30248874.1075857584813.JavaMail.e...,caroline.abramo@enron.com,Harvard Mgmt,3.0,53.0,enron.com,"Tue, 12 Dec 2000 09:33:00 -0800 (PST)",0.0
4,arnold-j,Message-ID: <2526360.1075857584903.JavaMail.ev...,klarnold@flash.net,Fwd: NYTimes.com Article: Suspended Rabbi Quit...,1.0,467.0,flash.net,"Mon, 11 Dec 2000 23:52:00 -0800 (PST)",1.0
...,...,...,...,...,...,...,...,...,...
517395,stf,Message-ID: <18618854.1075840028791.JavaMail.e...,jodi.droll@xcelenergy.com,RE: Scheduling Time constant,1.0,259.0,xcelenergy.com,"Thu, 11 Oct 2001 09:39:47 -0700 (PDT)",0.0
517396,stf,Message-ID: <14350892.1075840028690.JavaMail.e...,mark.hackney@aps.com,"FW: Standards Announcement - October 17, 2001",1.0,124.0,aps.com,"Wed, 17 Oct 2001 16:11:39 -0700 (PDT)",1.0
517397,stf,Message-ID: <21704474.1075840029683.JavaMail.e...,paul.rice@pacificorp.com,Late tags,1.0,20.0,pacificorp.com,"Fri, 6 Apr 2001 01:42:00 -0700 (PDT)",0.0
517398,stf,Message-ID: <9367927.1075840029633.JavaMail.ev...,gjcarter@bpa.gov,RE: BCHA Automatic Denial/Approval,18.0,167.0,bpa.gov,"Mon, 9 Apr 2001 23:11:00 -0700 (PDT)",0.0


In [8]:
def preprocess_text(text):
    # Tokenize
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens

# Apply preprocessing to EmailContent
df['processed_content'] = df['EmailContent'].apply(preprocess_text)

# Topic Modeling using LDA

In [9]:
# !pip3 install gensim
# !pip3 install scikit-learn

In [13]:
import gensim
from gensim import corpora
from gensim.models import LdaMulticore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [14]:
dictionary = corpora.Dictionary(df['processed_content'])
corpus = [dictionary.doc2bow(text) for text in df['processed_content']]

# Train LDA model
num_topics = 10  # Adjust as needed
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics)

# Get topic distributions for each email
topic_distributions = [lda_model.get_document_topics(doc) for doc in corpus]

In [15]:
def get_dominant_topic(topic_dist):
    return max(topic_dist, key=lambda x: x[1])[0]

df['dominant_topic'] = [get_dominant_topic(dist) for dist in topic_distributions]

In [17]:
df['dominant_topic'].unique()

array([8, 6, 2, 1, 0, 7, 9, 4, 5, 3])

In [18]:
df

,Employee,EmailContent,sender_email,subject,num_receivers,email_length,email_domain,sent_time,is_forwarded,processed_content,dominant_topic
0,arnold-j,Message-ID: <19171686.1075857585034.JavaMail.e...,slafontaine@globalp.com,re:summer inverses,1.0,29.0,globalp.com,"Fri, 8 Dec 2000 05:05:00 -0800 (PST)",0.0,"[thyme, date, fri, dec, pst, slafontaine, subj...",8
1,arnold-j,Message-ID: <29887033.1075857630725.JavaMail.e...,iceoperations@intcx.com,The WTI Bullet swap contracts,2.0,110.0,intcx.com,"Tue, 15 May 2001 09:43:00 -0700 (PDT)",0.0,"[thyme, date, tue, may, pdt, iceoperations, ic...",6
2,arnold-j,Message-ID: <29084893.1075849630138.JavaMail.e...,jeff.youngflesh@enron.com,Invitation: EBS/GSS Meeting w/Bristol Babcock ...,4.0,17.0,enron.com,"Mon, 27 Nov 2000 01:49:00 -0800 (PST)",0.0,"[thyme, date, mon, nov, pst, subject, invitati...",2
3,arnold-j,Message-ID: <30248874.1075857584813.JavaMail.e...,caroline.abramo@enron.com,Harvard Mgmt,3.0,53.0,enron.com,"Tue, 12 Dec 2000 09:33:00 -0800 (PST)",0.0,"[thyme, date, tue, dec, pst, subject, harvard,...",6
4,arnold-j,Message-ID: <2526360.1075857584903.JavaMail.ev...,klarnold@flash.net,Fwd: NYTimes.com Article: Suspended Rabbi Quit...,1.0,467.0,flash.net,"Mon, 11 Dec 2000 23:52:00 -0800 (PST)",1.0,"[thyme, date, mon, dec, pst, klarnold, subject...",1
...,...,...,...,...,...,...,...,...,...,...,...
517395,stf,Message-ID: <18618854.1075840028791.JavaMail.e...,jodi.droll@xcelenergy.com,RE: Scheduling Time constant,1.0,259.0,xcelenergy.com,"Thu, 11 Oct 2001 09:39:47 -0700 (PDT)",0.0,"[thyme, date, thu, oct, pdt, isas, subject, sc...",2
517396,stf,Message-ID: <14350892.1075840028690.JavaMail.e...,mark.hackney@aps.com,"FW: Standards Announcement - October 17, 2001",1.0,124.0,aps.com,"Wed, 17 Oct 2001 16:11:39 -0700 (PDT)",1.0,"[thyme, date, wed, oct, pdt, isas, subject, fw...",4
517397,stf,Message-ID: <21704474.1075840029683.JavaMail.e...,paul.rice@pacificorp.com,Late tags,1.0,20.0,pacificorp.com,"Fri, 6 Apr 2001 01:42:00 -0700 (PDT)",0.0,"[thyme, date, fri, apr, pdt, isas, subject, la...",5
517398,stf,Message-ID: <9367927.1075840029633.JavaMail.ev...,gjcarter@bpa.gov,RE: BCHA Automatic Denial/Approval,18.0,167.0,bpa.gov,"Mon, 9 Apr 2001 23:11:00 -0700 (PDT)",0.0,"[thyme, date, mon, apr, pdt, gjcarter, khara, ...",0


In [ ]:
# Save the updated dataframe to a new CSV file
df.to_csv('./data/LDA_output/LDA_topics_modelled_emails.csv', index=False)